# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import copy

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read in cities.csv file to data frame
weather_data = pd.read_csv("output_data/cities.csv")
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Goderich,43.7501,-81.7165,35.60,55,100,4.61,CA,2021-04-20 17:04:15
1,Chala,-7.5833,31.2667,60.80,94,75,5.75,TZ,2021-04-20 17:04:15
2,Hofn,64.2539,-15.2082,36.91,43,14,7.81,IS,2021-04-20 17:02:30
3,Butaritari,3.0707,172.7902,82.11,76,91,18.61,KI,2021-04-20 17:01:27
4,Hermanus,-34.4187,19.2345,60.01,90,100,5.01,ZA,2021-04-20 17:01:12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[['Lat', 'Lng']]

# Store Humidity
humidity = weather_data['Humidity'].astype(float)

In [4]:
# Customize the size of the figure
figure_layout = {'width': '600px',
                'height': '500px',
                'border': '1px solid black',
                'padding': '1px',
                'margin': '0 auto 0 auto'
                }
fig = gmaps.figure(layout=figure_layout)

# Create a humidity Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300,
                                 point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = weather_data.loc[(weather_data['Max Temp'] < 80) & (weather_data['Max Temp'] > 70)
                               & (weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0)]
ideal_weather.dropna()
ideal_weather.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
124,Oussouye,12.4850,-16.5469,75.2,83,0,6.91,SN,2021-04-20 17:04:53
191,Maceió,-9.6658,-35.7353,77.0,88,0,4.61,BR,2021-04-20 17:04:02
328,Itakyry,-24.9333,-55.2167,73.4,64,0,2.30,PY,2021-04-20 17:06:03
409,Belmonte,-15.8631,-38.8828,77.0,83,0,5.75,BR,2021-04-20 17:02:05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = copy.deepcopy(ideal_weather)
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
124,Oussouye,12.4850,-16.5469,75.2,83,0,6.91,SN,2021-04-20 17:04:53,
191,Maceió,-9.6658,-35.7353,77.0,88,0,4.61,BR,2021-04-20 17:04:02,
328,Itakyry,-24.9333,-55.2167,73.4,64,0,2.30,PY,2021-04-20 17:06:03,
409,Belmonte,-15.8631,-38.8828,77.0,83,0,5.75,BR,2021-04-20 17:02:05,


In [7]:
# Set up a parameters dictionary
params = {
          'radius': 5000,
          'types': 'lodging',
          'keyword': 'hotel',
          'key': g_key
         }
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    # Get lat, lng from df
    lat = row['Lat']
    lng = row['Lng']

    # Change location each iteration while leaving original params in place
    params['location'] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print(f"{row['City']} not found skipping...") 
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
124,Oussouye,12.4850,-16.5469,75.2,83,0,6.91,SN,2021-04-20 17:04:53,Oussouye Bed&Breakfast
191,Maceió,-9.6658,-35.7353,77.0,88,0,4.61,BR,2021-04-20 17:04:02,Jatiúca Hotel & Resort
328,Itakyry,-24.9333,-55.2167,73.4,64,0,2.30,PY,2021-04-20 17:06:03,Posada Santa Celina
409,Belmonte,-15.8631,-38.8828,77.0,83,0,5.75,BR,2021-04-20 17:02:05,Pousada do Diogo


In [8]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Customize size of the figure
figure_layout = {'width': '600px',
                'height': '500px',
                'border': '1px solid black',
                'padding': '1px',
                'margin': '0 auto 0 auto'
                }
fig = gmaps.figure(layout=figure_layout)

# Add heat layer
fig.add_layer(heat_layer)

# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…